# 🦷 16-Class Tooth Segmentation - 250 Epochs
**Complete training with all fixes applied**

## 📦 Step 1: Install Dependencies

In [ ]:
!pip install -q roboflow opencv-python-headless
print('✓ Dependencies installed')

## 📥 Step 2: Download Dataset

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key='O0qh97MHEruEAi4iUdAl')
project = rf.workspace('mv-dsnw8').project('segmentation-dental')
version = project.version(1)
dataset = version.download('coco-segmentation')
print(f'✓ Dataset downloaded to: {dataset.location}')

## 📚 Step 3: Import Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T

import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import json, os
from tqdm import tqdm
import time

print(f'PyTorch: {torch.__version__}')
print(f'CUDA: {torch.cuda.is_available()}')

## 🏗️ Step 4: Define U-Net Model

In [ ]:
class UNet(nn.Module):
    def __init__(self, num_classes, in_channels=3, base_channels=64):
        super().__init__()
        self.enc1 = self.conv_block(in_channels, base_channels)
        self.enc2 = self.conv_block(base_channels, base_channels*2)
        self.enc3 = self.conv_block(base_channels*2, base_channels*4)
        self.enc4 = self.conv_block(base_channels*4, base_channels*8)
        self.bottleneck = self.conv_block(base_channels*8, base_channels*16)
        self.upconv4 = nn.ConvTranspose2d(base_channels*16, base_channels*8, 2, stride=2)
        self.dec4 = self.conv_block(base_channels*16, base_channels*8)
        self.upconv3 = nn.ConvTranspose2d(base_channels*8, base_channels*4, 2, stride=2)
        self.dec3 = self.conv_block(base_channels*8, base_channels*4)
        self.upconv2 = nn.ConvTranspose2d(base_channels*4, base_channels*2, 2, stride=2)
        self.dec2 = self.conv_block(base_channels*4, base_channels*2)
        self.upconv1 = nn.ConvTranspose2d(base_channels*2, base_channels, 2, stride=2)
        self.dec1 = self.conv_block(base_channels*2, base_channels)
        self.outconv = nn.Conv2d(base_channels, num_classes, 1)
        self.pool = nn.MaxPool2d(2, 2)
    
    def conv_block(self, in_ch, out_ch):
        return nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1), nn.BatchNorm2d(out_ch), nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1), nn.BatchNorm2d(out_ch), nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool(enc1))
        enc3 = self.enc3(self.pool(enc2))
        enc4 = self.enc4(self.pool(enc3))
        bottleneck = self.bottleneck(self.pool(enc4))
        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat([dec4, enc4], dim=1)
        dec4 = self.dec4(dec4)
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat([dec3, enc3], dim=1)
        dec3 = self.dec3(dec3)
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat([dec2, enc2], dim=1)
        dec2 = self.dec2(dec2)
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat([dec1, enc1], dim=1)
        dec1 = self.dec1(dec1)
        return self.outconv(dec1)

print('✓ U-Net defined')

## 📊 Step 5: Define Dataset Class (With Bulletproof Error Handling)

In [ ]:
from collections import defaultdict

class CocoSegmentationDataset(Dataset):
    def __init__(self, img_dir, ann_file, img_size=256, transforms=None):
        self.img_dir = img_dir
        self.img_size = img_size
        self.transforms = transforms
        with open(ann_file, 'r') as f:
            self.coco_data = json.load(f)
        self.images = {img['id']: img for img in self.coco_data['images']}
        self.categories = {cat['id']: cat for cat in self.coco_data['categories']}
        unique_cat_ids = sorted(self.categories.keys())
        self.cat_id_to_idx = {cat_id: idx for idx, cat_id in enumerate(unique_cat_ids)}
        self.num_classes = len(unique_cat_ids)
        print(f'✓ {self.num_classes} classes, valid range: [0, {self.num_classes-1}]')
        self.img_to_anns = defaultdict(list)
        for ann in self.coco_data['annotations']:
            self.img_to_anns[ann['image_id']].append(ann)
        self.img_ids = list(self.images.keys())
    
    def __len__(self):
        return len(self.img_ids)
    
    def __getitem__(self, idx):
        img_id = self.img_ids[idx]
        img_info = self.images[img_id]
        img_path = os.path.join(self.img_dir, img_info['file_name'])
        image = Image.open(img_path).convert('RGB')
        h, w = img_info['height'], img_info['width']
        mask = np.zeros((h, w), dtype=np.uint8)
        anns = self.img_to_anns[img_id]
        for ann in anns:
            if 'segmentation' in ann:
                cat_id = ann['category_id']
                if cat_id not in self.cat_id_to_idx:
                    continue
                mapped_id = self.cat_id_to_idx[cat_id]
                for seg in ann['segmentation']:
                    if len(seg) < 6 or len(seg) % 2 != 0:
                        continue
                    try:
                        poly = np.array(seg).reshape(-1, 2).astype(np.int32)
                        cv2.fillPoly(mask, [poly], mapped_id)
                    except (ValueError, cv2.error):
                        continue
        mask = np.clip(mask, 0, self.num_classes - 1)
        mask_pil = Image.fromarray(mask)
        image = image.resize((self.img_size, self.img_size))
        mask_pil = mask_pil.resize((self.img_size, self.img_size), Image.NEAREST)
        if self.transforms:
            image = self.transforms(image)
        mask_tensor = torch.from_numpy(np.array(mask_pil)).long()
        assert mask_tensor.max() < self.num_classes, f'Invalid mask value'
        return image, mask_tensor

print('✓ Dataset class defined')

## 📂 Step 6: Set Dataset Paths

In [ ]:
DATASET_ROOT = 'Segmentation-Dental-1'
TRAIN_IMG_DIR = f'{DATASET_ROOT}/train'
TRAIN_JSON = f'{DATASET_ROOT}/train/_annotations.coco.json'
VAL_IMG_DIR = f'{DATASET_ROOT}/valid'
VAL_JSON = f'{DATASET_ROOT}/valid/_annotations.coco.json'
print('✓ Paths set')

## ⚙️ Step 7: Hyperparameters (250 Epochs)

In [ ]:
EPOCHS = 250
BATCH_SIZE = 8
LEARNING_RATE = 5e-4
IMG_SIZE = 256
MODEL_SAVE_PATH = 'best_unet_250epochs.pt'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {DEVICE}, Epochs: {EPOCHS}')

## 📊 Step 8: Create Data Loaders

In [ ]:
train_transform = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)), T.RandomHorizontalFlip(p=0.5),
    T.RandomVerticalFlip(p=0.3), T.RandomRotation(degrees=15),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1),
    T.ToTensor(), T.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225]),
])
val_transform = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)), T.ToTensor(),
    T.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225]),
])

train_dataset = CocoSegmentationDataset(TRAIN_IMG_DIR, TRAIN_JSON, IMG_SIZE, train_transform)
val_dataset = CocoSegmentationDataset(VAL_IMG_DIR, VAL_JSON, IMG_SIZE, val_transform)
NUM_CLASSES = train_dataset.num_classes

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print(f'✓ NUM_CLASSES: {NUM_CLASSES}')
print(f'✓ Train: {len(train_dataset)}, Val: {len(val_dataset)}')

## 🧠 Step 9: Initialize Model

In [ ]:
model = UNet(num_classes=NUM_CLASSES).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
print(f'✓ Model initialized with {NUM_CLASSES} classes')

## 🚀 Step 10: Training Loop (250 Epochs)

In [ ]:
best_val_loss = float('inf')
train_losses, val_losses, train_accs, val_accs = [], [], [], []
start_time = time.time()

print('='*70)
print(f'TRAINING FOR {EPOCHS} EPOCHS')
print('='*70)

for epoch in range(EPOCHS):
    model.train()
    train_loss, train_correct, train_total = 0, 0, 0
    for imgs, masks in tqdm(train_loader, desc=f'Epoch {epoch+1}/{EPOCHS} [Train]'):
        imgs, masks = imgs.to(DEVICE), masks.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        pred = outputs.argmax(1)
        train_correct += (pred == masks).sum().item()
        train_total += masks.numel()
    
    avg_train_loss = train_loss / len(train_loader)
    train_acc = 100 * train_correct / train_total
    
    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0
    with torch.no_grad():
        for imgs, masks in tqdm(val_loader, desc=f'Epoch {epoch+1}/{EPOCHS} [Val]  '):
            imgs, masks = imgs.to(DEVICE), masks.to(DEVICE)
            outputs = model(imgs)
            loss = criterion(outputs, masks)
            val_loss += loss.item()
            pred = outputs.argmax(1)
            val_correct += (pred == masks).sum().item()
            val_total += masks.numel()
    
    avg_val_loss = val_loss / len(val_loader)
    val_acc = 100 * val_correct / val_total
    scheduler.step()
    
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
    
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        print(f'✓ Best model: {avg_val_loss:.4f}')
    
    if (epoch + 1) % 10 == 0:
        elapsed = time.time() - start_time
        print(f'\nEpoch [{epoch+1}/{EPOCHS}]')
        print(f'  Train: {avg_train_loss:.4f} | {train_acc:.2f}%')
        print(f'  Val:   {avg_val_loss:.4f} | {val_acc:.2f}%')
        print(f'  Time: {elapsed/60:.1f}min')

print(f'\n✓ Training complete: {(time.time()-start_time)/3600:.2f}h')
print(f'✓ Best val loss: {best_val_loss:.4f}')
print(f'✓ Final train acc: {train_accs[-1]:.2f}%')
print(f'✓ Final val acc: {val_accs[-1]:.2f}%')

## 📈 Step 11: Plot Results

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5))
axes[0].plot(train_losses, label='Train')
axes[0].plot(val_losses, label='Val')
axes[0].set_title('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[1].plot(train_accs, label='Train')
axes[1].plot(val_accs, label='Val')
axes[1].set_title('Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
plt.show()

## 📥 Step 12: Download Model

In [ ]:
from google.colab import files
files.download(MODEL_SAVE_PATH)
print(f'✓ Downloaded: {MODEL_SAVE_PATH}')